In [1]:
!pip install "pydantic>=2.0,<2.12.3" "gradio>=5.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.6/460.6 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.12.3
    Uninstalling pydantic-2.12.3:
      Successfully uninstalled pydantic-2.12.3


In [2]:
!pip install -U --quiet \
    langchain \
    langchain-community \
    langchain-openai \
    langchain-huggingface \
    langchain-text-splitters \
    faiss-cpu \
    pypdf \
    sentence-transformers \
    pydantic


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.6/463.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sourc

In [3]:
import os
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_classic.chains import RetrievalQA
from langchain_classic.prompts import PromptTemplate

In [4]:
###################################---Loading Documents---#######################################

In [5]:
documents = []

for file in os.listdir():
    if file.endswith(".pdf"):
        loader = PyPDFLoader(file)
        documents.extend(loader.load())
    elif file.endswith(".txt"):
        loader = TextLoader(file)
        documents.extend(loader.load())

print(f"Loaded {len(documents)} documents")

Loaded 2 documents


In [6]:
###################################---Turning into Chunks---#######################################

In [7]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=100,
)

In [8]:
chunks =splitter.split_documents(documents)
print(f"Created {len(chunks)} chunks")
print(f"The first chunk:\n{chunks[0]}")

Created 108 chunks
The first chunk:
page_content='There's no single test to specifically diagnose polycystic ovary syndrome (PCOS). Your health care provider is likely to start with a discussion of your symptoms, medications and any other medical conditions. Your provider also may ask about your menstrual periods and any weight changes. A physical exam includes checking for signs of excess hair growth, insulin resistance and acne.

Your health care provider might then recommend:' metadata={'source': 'pcos-diagnosisAndTreatment.txt'}


In [9]:
###################################---Creating Embedding Model---#######################################

In [10]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 93be8324-23b1-4293-b86c-23837712157b)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 03703aae-caf1-42f6-91d4-c91d165fb080)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 2s [Retry 2/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 66ded1ea-6953-4155-92ed-82b1eec32c34)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 4s [Retry 3/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co'

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
###################################---Storing embeddings in vector database---#######################################

In [12]:
vector_db = FAISS.from_documents(chunks, embedding_model)

In [13]:
retriever = vector_db.as_retriever(search_kwargs={"k": 3})

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "Token Key"

In [15]:
!pip install -q transformers

In [16]:
###################################---Adding LLM---#######################################

In [17]:
from langchain_classic.llms import HuggingFacePipeline
from transformers import pipeline

# Use text2text generation model (Flan-T5)
pipe = pipeline(
    "text2text-generation",
    model="google/flan-t5-large",
    tokenizer="google/flan-t5-large",
    max_length=512
)

llm = HuggingFacePipeline(pipeline=pipe)

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 7d02e056-2da3-40ba-b48e-65289d17f211)')' thrown while requesting HEAD https://huggingface.co/google/flan-t5-large/resolve/main/spiece.model
Retrying in 1s [Retry 1/5].


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 4ef45894-0e3d-44c1-9b16-eb5e35ad1164)')' thrown while requesting HEAD https://huggingface.co/google/flan-t5-large/resolve/main/tokenizer.json
Retrying in 1s [Retry 1/5].


tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
/tmp/ipython-input-2729106954.py:12: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [18]:
###################################---Adding Custom Propmts---#######################################

In [19]:
prompt_template = """
You are a health education chatbot.

Using ONLY the information from the context below,
rewrite the answer as a clear, complete, well-formed sentence.
Do NOT copy phrases directly.
Do NOT answer in bullet points.
If the answer is not in the context, say:
"I do not have enough knowledge to answer that."

Question:
{question}

Context:
{context}

Final Answer (one complete sentence):
"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)


In [20]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

In [21]:
###################################---Checking Answer---#######################################

In [22]:
query = "What are the main symptoms of PCOS?"
result = qa_chain(query)

print("Answer:\n", result["result"])

/tmp/ipython-input-708095668.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  result = qa_chain(query)


Answer:
 PCOS has a wide variety of signs and symptoms. They include issues with ovulation (such as irregular periods), excess levels of androgens (hormones that trigger male characteristics, such as facial hair growth), and metabolism (such as weight gain).[3] Symptoms usually start in puberty, but may be masked if oral contraceptives are started early.[3]


In [23]:
query = "How to treat PCOS? Can we totally cure it?"
result = qa_chain(query)

print("Answer:\n", result["result"])

Answer:
 PCOS has no cure,[4] and management is focused on the relief of symptoms.[5] Treatment usually involves lifestyle changes such as diet and exercise of moderate intensity.[45] Metabolic issues can further be treated with metformin or GLP-1 receptor agonists. For women with a BMI over 35, bariatric surgery may be a further option.[11] Combined oral contraceptives are especially effective and used as the first line of treatment to reduce acne and hirsutism and regulate the menstrual cycle.[46] Other typical acne treatments and hair removal techniques may be used.[9] First-line treatment for Treatment of PCOS shows no to moderate effect on depression or anxiety, and standard therapies (such a psychotherapy and anti-depressants) are recommended instead.[11] Cognitive behaviour therapy can be used for girls and women with low self-esteem, poor body image, disordered eating or psychosexual dysfunction.[71] It can be difficult to become pregnant with PCOS due to irregular ovulation. T

In [24]:
query = "What is typhoid?"
result = qa_chain(query)

print("Answer:\n", result["result"])

Answer:
 I do not have enough knowledge to answer that.
